In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import datetime

In [2]:
# Add in stocks from the selected Portfolio
# stocks = ['^SPX','AMP','SCHW','BRK-B','AAPL','STLA','YUM','ADM','AMR','PWR','XOM','CE','ELV','DGX','REGN','MRK','AMAT','GOOG','JBL','ADI']
stocks = ['^SPX', 'INTC','AMD','NVDA']

In [3]:
# Define the time period
end_date = datetime.datetime.now()
start_date = end_date - datetime.timedelta(days=5*365)

In [4]:
# Initialize a dictionary to hold ticker data
stock_data = {}

# Fetch the monthly closing prices for each stock
for ticker in stocks:
    stock = yf.Ticker(ticker)
    df = stock.history(start=start_date, end=end_date, interval="1mo")
    stock_data[ticker] = df['Close']

# Combine the data into a single DataFrame
combined_data = pd.DataFrame(stock_data)

# Clean up the DataFrame by dropping rows with NaN values that can occur at the end
# combined_data.dropna(how='all', inplace=True)

In [5]:
combined_data.head()

,^SPX,INTC,AMD,NVDA
Date,,,,
2019-06-01 00:00:00-04:00,2941.760010,41.977592,30.370001,40.825207
2019-07-01 00:00:00-04:00,2980.379883,44.327705,30.450001,41.941357
2019-08-01 00:00:00-04:00,2926.459961,41.574215,31.450001,41.640568
2019-09-01 00:00:00-04:00,2976.739990,45.492168,28.990000,43.314133
2019-10-01 00:00:00-04:00,3037.560059,49.906303,33.930000,50.020123


In [6]:
# Calculate excess the month to month excess returns for each stock and store it in a new dataframe
excess_ret = combined_data.pct_change()
excess_ret.tail()

,^SPX,INTC,AMD,NVDA
Date,,,,
2024-01-01 00:00:00-05:00,0.015896,-0.142686,0.137575,0.242527
2024-02-01 00:00:00-05:00,0.051721,-0.000697,0.148130,0.285809
2024-03-01 00:00:00-05:00,0.031019,0.029024,-0.062536,0.142128
2024-04-01 00:00:00-04:00,-0.041615,-0.310165,-0.122500,-0.043715
2024-05-01 00:00:00-04:00,0.037133,-0.016369,-0.040788,0.040231


In [31]:
stocks.__len__()

3

In [10]:
# Add a row for total portfolio
for givenRow in excess_ret:
    
    excess_ret.loc[givenRow, "Portfolio"] = 

SyntaxError: invalid syntax (742023493.py, line 4)

In [19]:
# Optional // If a stock has NaN values find out where they begin
fst = combined_data['AMR'].first_valid_index()
ind = combined_data.index.get_loc(fst)
combined_data.iloc[(ind-3):(ind+3)]

KeyError: 'AMR'

In [7]:
# Separate the S&P 500 returns and the stocks' returns
spx_returns = excess_ret.iloc[:, 0]
stocks_returns = excess_ret.iloc[:, 0:]

# Create an empty DataFrame to store the analysis results
perf_metrics = pd.DataFrame(index=excess_ret.columns)

# Calculate each metric
perf_metrics['Mean Annualized Return'] = stocks_returns.mean() * 12
perf_metrics['Annualized Std Dev'] = stocks_returns.std() * np.sqrt(12)

In [8]:
perf_metrics

,Mean Annualized Return,Annualized Std Dev
^SPX,0.133577,0.181429
INTC,-0.003096,0.361492
AMD,0.464646,0.533678
NVDA,0.761696,0.491004


In [9]:
# Market variance for Beta calculations
market_variance = spx_returns.var() * 12

# Calculate metrics that require row-wise operations
for stock in stocks_returns:
    stock_returns = stocks_returns[stock]
    cov_with_market = stock_returns.cov(spx_returns) * 12  # Annualize the covariance
    beta = cov_with_market / market_variance                                                                   
    
    perf_metrics.loc[stock, 'SPX Correlation'] = stock_returns.corr(spx_returns)
    perf_metrics.loc[stock, 'Beta'] = beta
    perf_metrics.loc[stock, 'Total Variance'] = stock_returns.var() * 12  # Annualize the variance         
    perf_metrics.loc[stock, 'Systematic Variance'] = beta ** 2 * market_variance                           
    perf_metrics.loc[stock, 'Unique Variance'] = perf_metrics.loc[stock, 'Total Variance'] - perf_metrics.loc[stock, 'Systematic Variance']
    perf_metrics.loc[stock, 'R-squared'] = perf_metrics.loc[stock, 'SPX Correlation'] ** 2 

# perf_metrics.format({})

In [12]:
perf_metrics       # what would you call this set of statistics? performance measures, can add sharpe ratio, sortino ratio, studdtzer index, maximum drawdown, sometimes called risk adjusted performance metrics, like sharpe ratio and  sortino this means they dont change with leverage

,Mean Annualized Return,Annualized Std Dev,SPX Correlation,Beta,Total Variance,Systematic Variance,Unique Variance,R-squared
^SPX,0.133577,0.181429,1.000000,1.000000,0.032916,0.032916,1.387779e-17,1.000000
INTC,-0.003096,0.361492,0.520013,1.036115,0.130677,0.035337,9.533999e-02,0.270414
AMD,0.464646,0.533678,0.573602,1.687269,0.284812,0.093709,1.911031e-01,0.329020
NVDA,0.761696,0.491004,0.603187,1.632421,0.241085,0.087715,1.533700e-01,0.363835


In [13]:
# Set float format for display
pd.options.display.float_format = '{:.6f}'.format

In [14]:
# Transpose the df
stks_anl_results_T = perf_metrics.transpose()
stks_anl_results_T

,^SPX,INTC,AMD,NVDA
Mean Annualized Return,0.133577,-0.003096,0.464646,0.761696
Annualized Std Dev,0.181429,0.361492,0.533678,0.491004
SPX Correlation,1.000000,0.520013,0.573602,0.603187
Beta,1.000000,1.036115,1.687269,1.632421
Total Variance,0.032916,0.130677,0.284812,0.241085
Systematic Variance,0.032916,0.035337,0.093709,0.087715
Unique Variance,0.000000,0.095340,0.191103,0.153370
R-squared,1.000000,0.270414,0.329020,0.363835
